#  Formatting 

<ol>
<li> All questions for a part gathered in the top for overview, and then each individual question placed as they are answered. 
<li> Individual questions are bolded, the answers are in regular text.
<li> Figures are followed by cell including the description of the figure.
<li> References are added at the end
<li> The following files are included in the zip folder attached to this file:
    <ol>
        <li>Folder with characters' txt files: <code>characters.zip/</code>
        <li>Folder with pre-processed tokenized text for each character: <code>preprocessed_texts.zip/</code>
        <li>LabMT word list: <code>Data_Set_S1.txt/</code>
    </ol>
</ol>

# Imports

In [ ]:
import urllib
import re
import pandas as pd
import io
import requests
from pathlib import Path
import numpy as np
import networkx as nx
from zipfile import ZipFile
from fa2 import ForceAtlas2
import matplotlib.pyplot as plt
import powerlaw
from typing import List
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import rbo 
from scipy.stats import ttest_ind, normaltest
import community as community_louvain
import json
import os
import regex as re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from wordcloud import WordCloud, ImageColorGenerator
import community
from community import community_louvain
import math
import seaborn as sns
from tqdm.notebook import tqdm
%matplotlib inline

# Part 0: Building the network 

To create our network, we downloaded the Zelda Wiki pages for all characters in BotW (during Week 4) and linked them via the hyperlinks connecting pages to each other. To achieve this goal we have used regular expressions!

> * Explain the strategy you have used to extract the hyperlinks from the Wiki-pages, assuming that you have already collected the pages with the Zelda API.
> * Show the regular expression(s) you have built and explain in details how it works.

In [ ]:
#Extract the pages with the characters' description from the Zelda API from characters.zip
#Extract all the files from the zip file included in the folder
with ZipFile("characters.zip", 'r') as zipFile:
    zipFile.extractall()
    
path = "characters/"

In [ ]:
#Read the ZeldaBotW_nodes_w_attributes_enriched.CSV file into a Pandas df
url= "https://raw.githubusercontent.com/SocialComplexityLab/socialgraphs2021/main/files/ZeldaBotW_nodes_w_attributes_enriched.csv"
df = pd.read_csv(url)
#Add a column with the word count of each character page
df['Word_count'] = [len(re.findall(r'\w+',open(path+word+".txt","r").read())) if word != 'Granté' else 0 for word in df['Name']]

In [ ]:
links = []

#For each character in the csv, extract the links to the nodes in the descripton file
path = "characters/"

for node in df["Name"]:
    with open(path+node+".txt","r") as f:
        node_description = f.read()
    #hypyerlink regex that link to another node of the network (e.g. another character). 
    pattern1 = r'\[\[(.*?)(?:\|.*?)?\]\]'
    pattern2 = r'\{\{(?:Plural|Term)\|BotW\|(.*?)(?:\|display.*?)?(\|link)?(?:\|display.*?)?\}\}'
    #Find patterns of two types and filter non-links using regex patterns
    links_pattern1 = list(re.findall(pattern1, node_description))
    links_pattern2 = [l[0] for l in re.findall(pattern2, node_description) if "|link" in l]
    #Append them to a list
    #Check if the matches are valid by comparing them with the names in the characters' list
    links_total = [l for l in set(links_pattern1+links_pattern2) if l in df.Name.values]
    links.append(links_total)

In [ ]:
#Create NetworkX DiGraph to store the network
G = nx.DiGraph()
nodes_with_attributes = [(df.Name[i],{'Role':df.Role[i],'Race':df.Race[i],'Gender':df.Gender[i], 'Word_count':df.Word_count[i]}) for i in df.index]
#First, add the nodes with the attributes
G.add_nodes_from(nodes_with_attributes)
#Second, add the edges, represented by the list of hyperlinks for each node
list_dict = {df.Name.values[i]: x for i,x in enumerate(links) for y in x if (y in df.Name.values)}
#Remove self references
G.add_edges_from([(x,i) for x in list_dict for i in list_dict[x] if x!=i ] )
print(f'num nodes: {G.number_of_nodes()}')
print(f'num edges: {G.number_of_edges()}')

In [ ]:
#Remove isolated nodes from the network
G.remove_nodes_from(list(nx.isolates(G)))
# Extract the largest component LC
LC = max(nx.weakly_connected_components(G),key=len)
#Save only the largest component as G
GG = nx.DiGraph(G.subgraph(LC))
#Undirected Graph
G_undirected = GG.to_undirected()

In [ ]:
print(f'num nodes after removing isolates: {G.number_of_nodes()}')
print(f'num edges after removing isolates: {G.number_of_edges()}')

> **Explain the strategy you have used to extract the hyperlinks from the Wiki-pages, assuming that you have already collected the pages with the Zelda API.**


The overall strategy for building is the following: 
<ol>
<li> We extracted the pages downloaded for each character from the zip file included in the folder. Each page corresponds to a character, which is a node in the network. 
<li> We used the <a href="https://raw.githubusercontent.com/SocialComplexityLab/socialgraphs2021/main/files/ZeldaBotW_nodes_w_attributes_enriched.csv">csv file</a>
    provided on Github  to find the Name, Race, Role and Gender of a character.
<li>Then, we found all the hyperlinks in a character's page that link to another node of the network (e.g. another character). We use regular expressions to extract all outgoing links from each of the pages you downloaded above. For each link, we checked if the target was a character. If yes, we kept it. If no, we discarded it.
<li> To build the network, we used a NetworkX DiGraph to store the network. We also stored the properties of the nodes (i.e. Role, Race, and Gender, Word Count). By storing this information as nodal features, we keep all the relevant data in one place.
<li> To clean the network, we removed the isolated nodes (ie. nodes that do not have any edge) from the network.
<li> Additionally, we extracted the largest component to make visualizations much easier. 
<li> For the sake of the visualisation, we converted our network to an undirected graph.
</ol>


> **Show the regular expression(s) you have built and explain in details how it works.**

In our approach, we used 2 regular expressions to extract all outgoing links from each of the Wiki pages:
<ol>
    <li>The first pattern <code>r'\[\[(.*?)(?:\|.*?|\#Breath of the Wild)?\]\]'</code> is used to extract direct links (i.e. <code>[[name of character]]</code>) and links to sections of another page (i.e. <code>[[Link#Breath of the Wild]]</code> or links using text (i.e <code>[[Princess Zelda|Zelda]]</code>) that contain square brackets. It is used to match text where: 
     <ol>  
        <li> There are two starting square brackets captured by <code>\[\[</code> 
        <li> Followed by a text group formed of any character (except for line terminators) and repeated as few times as possible, captured by <code>(.*?)</code> 
            <ol>  
             <li> <code>.</code> matches any character (except for line terminators)
             <li> <code>*?</code> matches the previous token between zero and unlimited times, as few times as possible, expanding as needed
            </ol>
         <li>Followed by EITHER a vertical bar and a text group OR the phrase <code>#Breath of the Wild</code> OR none of these, captured by a non-capturing group <code>(?:\|.*?|\#Breath of the Wild)</code>
             <ol>  
                 <li> <code>?:</code> defines the non-capturing group matches any character (except for line terminators)
                 <li> <code>\|.*?</code> matches any character delimited by a vertical line and followed by any character (<code>.</code>), which is matched between zero and unlimited times, as few times as possible, expanding as needed (<code>*?</code>)
                 <li><code>|</code> the vertical bar is used to define alternation, which is an OR statement between multiple expressions.
                 <li><code>\#Breath of the Wild</code> matches the text <code>#Breath of the Wild</code>
             </ol>
             <li>Followed by 2 ending square brackets <code>\]\]</code>
     </ol>
    <li>The second pattern <code>r'\{\{(?:Plural|Term)\|BotW\|(.*?)(?:\|display.*?)?(\|link)?(?:\|display.*?)?\}\ </code> is used to identify links using plural terms and links to pages that have curly parentheses (ie. <code>{Term|Plural|Princess Zelda|link|display=Zelda}</code>). It is used to match text where: 
        <ol>  
            <li>There are two starting curly brackets captured by <code>\{\{</code> 
            <li>Followed by EITHER the words <code>Plural</code> OR <code>Term</code> OR neither captured by a non-capturing group <code>(?:Plural|Term)</code>
            <li>Followed by a vertical bar and the word <code>BotW</code> followed by another vertical bar, captured by <code>\|BotW\|</code> 
            <li>Followed by a group formed of any character AND the group <code>| display</code>, if any, and the group <code>| link</code>, if any, repeated at most once, and the group <code>| display</code> again, if any at all captured by <code>(.*?)(?:\|display.*?)?(\|link)?(?:\|display.*?)</code>.
                <ol> 
                     <li><code>.</code> matches any character (except for line terminators)
                     <li> <code>*?</code> matches the previous token between zero and unlimited times, as few times as possible, expanding as needed 
                    <li><code>(?:\|display.*?)</code> identifies the characters included in the group but does not capture the group. It is used to match the links which have the name <code>display</code> in the link (ie. <code>{{Term|Series|Princess Zelda|link|display=Zelda}}</code>)
                    <li><code>?</code> matches the previous token between zero and one times, as many times as possible, giving back as needed 
                    <li><code>(\|link)</code> identifies the <code>link</code> text in hyperlinks such as <code>{{Term|BotW|Sheikah|link}</code>.        
               </ol> 
             <li>Followed by 2 ending curly brackets <code>\}\}</code>
        </ol>
</ol>


# Part 1: Network visualization and basic stats

Visualize the network (from lecture 5) and calculate stats (from lecture 4 and 5). For this exercise, we assume that you've already generated the BotW network and extracted the giant connected component. Use the GCC to report the results.

Exercise 1a: Stats (see lecture 4 and 5 for more hints)

> * What is the number of nodes in the network?
> * What is the number of links?
> * Who is the top connected character in BotW? (Report results for the in-degrees and out-degrees). Comment on your findings. Is this what you would have expected?
> * Who are the top 5 most connected allies (again in terms of in/out-degree)?
> * Who are the top 5 most connected enemies -- bosses included -- (again in terms of in/out-degree)?
> * Plot the in- and out-degree distributions.
>   * What do you observe?
>   * Can you explain why the in-degree distribution is different from the out-degree distribution?
> * Find the exponent of the degree distribution (by using the powerlaw package) for the in- and out-degree distribution. What does it say about our network?
> * Compare the degree distribution of the undirected graph to a random network with the same number of nodes and probability of connection p. Comment your results.

**_Exercise 1a_: Stats (see lecture 4 and 5 for more hints)**

> **What is the number of nodes in the network? What is the number of links?**

In [ ]:
#number of nodes
print(f'Nodes in graph: {len(list(G.nodes()))}')
print(f'Nodes in undirected graph: {len(list(G_undirected.nodes()))}')
#number of links
print(f'Links: {G.number_of_edges()}')


>**Who is the top connected character in BotW? (Report results for the in-degrees and out-degrees).**
>**Comment on your findings. Is this what you would have expected?**

In [ ]:
#list of in-degrees
in_deg = [v for k, v in dict(G.in_degree).items()]
#list of out-degrees
out_deg =  [v for k, v in dict(G.out_degree).items()]

In [ ]:
#sort the in and out-degree lists and extract the top value
top_character_in = sorted([(node, degree) for node, degree in G.in_degree], key=lambda t: t[1])[-1]
top_character_out = sorted([(node, degree) for node, degree in G.out_degree], key=lambda t: t[1])[-1]
print(f'The top connected character (in-degrees) is {top_character_in[0]} with {top_character_in[1]} links.')
print(f'The top connected character (out-degrees) is {top_character_out[0]} with {top_character_out[1]} links.')

The in-degree of node $i$ is the total number of connections onto node $i$, representing the number of other hyperlinks from other Wiki pages that point to a selected character. The top connected character in the in-degrees list is Link, with 396 links. This is expected, as Link is the main character, which is referenced by most of the other characters in their Wiki pages. 

On the other hand, the out-degree of node $i$ is the total number of connections coming from node $i$, representing the number of hyperlinks that point from the current character page to other character pages. The top connected character in the out-degrees list is Traysi, with 14 degrees. According to her Wiki page, Traysi is a Hylian investigative journalist who can be found researching her next story in Gerudo Town. It makes sense that she is the top connected character, because she references most of the other characters in her page, due to the nature of her role.

> **Who are the top 5 most connected allies (again in terms of in/out-degree)?**

In [ ]:
#sort the in and out-degree lists and extract the top 5 Allies from each
allies = [(node, degree) for node, degree in G.in_degree if G.nodes[node]['Role']=='Ally' ]
top_ally_in = sorted([(node, degree) for node, degree in allies], key=lambda t: t[1])[-5:]
top_ally_out = sorted([(node, degree) for node, degree in allies], key=lambda t: t[1])[-5:]
print(f'Top 5 most connected allies (in-degree):\n {top_ally_in}')
print(f'Top 5 most connected allies (outt-degree):\n {top_ally_out}')

> **Who are the top 5 most connected enemies -- bosses included -- (again in terms of in/out-degree)?**

In [ ]:
#sort the in and out-degree lists and extract the top 5 Ennemies from each
ennemies = [(node, degree) for node, degree in G.in_degree if (G.nodes[node]['Role']=='Ennemy' or  G.nodes[node]['Role']=='Boss')]
top_ennemy_in = sorted([(node, degree) for node, degree in ennemies], key=lambda t: t[1])[-5:]
top_ennemy_out = sorted([(node, degree) for node, degree in ennemies], key=lambda t: t[1])[-5:]
print(f'Top 5 most connected ennemies (in-degree):\n {top_ennemy_in}')
print(f'Top 5 most connected ennemies (out-degree):\n {top_ennemy_out}')

> **Plot the in- and out-degree distributions.**
>   **What do you observe?**


In [ ]:
#Function which plots both the lin-lin and log-loge degree distribution of a list of degrees
def plot_degree_distribution(degrees, title1, title2):
    sns.set()
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
    max_degree = np.max(degrees)
    min_degree = np.min(degrees)
    vector = np.arange(min_degree,max_degree)
    hist_pois, bin_edges = np.histogram(degrees,bins=vector)
    hist, bin_edges = np.histogram(degrees, bins=vector)
    bin_means = [0.5 * (bin_edges[i] + bin_edges[i+1]) for i in range(len(bin_edges)-1)]
    ax1.bar(bin_means, hist, width=bin_edges[1] - bin_edges[0], color='b', edgecolor='blue')
    ax1.set_title(title1)
    ax1.set_xlabel("Degree")
    ax1.set_ylabel("Frequency")
    
    plt.loglog(bin_means, hist, marker='.', linestyle='None')
    plt.title(title2)
    plt.xlabel("Degree")
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
#Plot degree distribution 
plot_degree_distribution(in_deg, "Fig.1. Histogram of in-degree distribution [Isolated]", 
                        "Fig.2. In-Degree Distribution (Log, Log) [Isolated]")

The linear and log-log in-degree distribution are shown in Figures 1 and 2 above. The histogram in Fig. 1 indicates that the in-degrees follow the power-law distribution. We notice that there are many nodes with only a few links (high frequency) and a few hubs (high degree) with large number of links. The linear dependence in the log-log plot in Fig. 2 supports the idea that indeed, the graph follows the power distribution.

In [ ]:
#Plot degree distribution 
plot_degree_distribution(out_deg, "Fig.3. Histogram of out-degree distribution [Isolated]", 
                        "Fig.4. Out-Degree Distribution (Log, Log) [Isolated]")

The linear and log-log out-degree distributions are shown in Figures 3 and 4 above. The histogram in Fig. 1 indicates that the out-degrees follows a distribution which resembles the Poisson distribution. The histogram in Fig. 3 indicates that some of the nodes have more or less the same number of links and that overall, we do not have many highly connected nodes. The log-log plot in Fig. 4 supports the idea that indeed, the graph follows a Poisson distribution.

>   **Can you explain why the in-degree distribution is different from the out-degree distribution?**

If we are to compare the in and out-degree distributions, there are a few differences to point out:
<ol> 
    <li>The in-degrees follow the power-law distribution, which places it in the scale-free regime, whereas the out-degrees follow a skewed Poisson distribution, which indicates that the out-degrees form a more radomized network.
    <li>Unlike the in-degrees, where most nodes have only a few links, out-degree nodes have comparable degrees and nodes with a large number of links are absent. This explains why the maximum frequency in Figure 1 spikes at 180, while the maximum frequency in Figure 3 is around 140.
</ol>

> **Find the exponent of the degree distribution (by using the `powerlaw` package) for the in- and out-degree distribution. What does it say about our network?**

In [ ]:
#Function which finds the degree values of a graph based on the values of a dataframe
def degree_values(dataframe, graph, direction:str=None, allies:bool=None, gender:str=None):
    ddf = dataframe
    
    if allies is not None:
        if allies is True:
            ddf = ddf[ddf["type"] == "Ally"]
        else:
            ddf = ddf[ddf["type"] != "Ally"]
    if gender is not None:
        ddf = ddf[ddf["Gender"] == gender]
    if direction is not None:
        if direction == "out":
            return [v for k, v in graph.out_degree if k in set(ddf["Name"])]
        else:
            return [v for k, v in graph.in_degree if k in set(ddf["Name"])]
    else:
        return [v for k, v in graph.to_undirected().degree if k in set(ddf["Name"])]

In [ ]:
#Use powerlaw.Fit(degree).alpha for fitting
degree_in = degree_values(df, G, direction="in")
degree_out = degree_values(df, G, direction="out")

#Find the degree exponent for in-, out- and total- degree distribution. 
fit_in = powerlaw.Fit(degree_in)
fit_out = powerlaw.Fit(degree_out)

print(f'Degree exponent for in-degree distribution: {fit_in.alpha}')
print(f'Degree exponent for out-degree distribution: {fit_out.alpha}')

The degree exponent or γ varies from system to system, prompting us to explore how the properties of a network change with γ ([1], Section 4.7). A value of γ higher than 2 indicates that a network is in the random regime. With a degree exponent for in-degree distribution is 3.13, and a degree exponent for out-degree distribution is 4.38, this is telling us that out-degree distribution has a higher degree of randomness compared to the in-degree distribution. 

Moreover, the degree exponent is an important indicator for the maximum degree of a scale-free network, which applies to the in-degree network. The equation below predicts that $k_{max}$ of the in-degree grows with the size of the network with exponent $1/(\gamma - 1)$:

$$k_{max}=k_{min}N^{\frac{1}{\gamma − 1}}      $$ 

For $N=540$, and an exponent $1/(\gamma - 1) \approx 0.5$ , $k_{max}$ is close to 23, which is reflected in Fig. 1 on the x-axis (ignoring the outliers). 

> **Compare the degree distribution of the undirected graph to a *random network* with the same number of nodes and probability of connection *p*. Comment your results.**

In [ ]:
#Create random network with the same number of nodes and same probability as the undirected graph
nodes = len(df)
degrees = [v for k, v in dict(G.degree).items()]
p = np.mean(degrees)/(nodes-1)
G_RN = nx.erdos_renyi_graph(n=nodes, p=p)

In [ ]:
deg = [v for k, v in dict(G_RN.degree).items()]
plot_degree_distribution(deg, "Fig.5. Histogram of Random Network", 
                        "Fig.6. Histogram of Random Network (Log, Log)")

Figures 5 and 6 describe the degree distributions of a random network with $N=540$ nodes and a probability $p=0.009$. The distributiton in Fig. 5 follows a Poisson distribution, which is confirmed by the log-log plot in Figure 6. We notice that the majority of nodes have the same number of links, with more nodes than in the distribution in Fig. 3.  In both Figure 3 and Figure 5 there are only a few highly connected nodes. 

_Exercise 1b_: Visualization (see lecture 5 for more hints)

> * Create a nice visualization of the total (undirected) network:
>   * Color nodes according to the role;
>   * Scale node-size according to degree;
>   * Get node positions based on the Force Atlas 2 algorithm;
>   * Whatever else you feel like that would make the visualization nicer.
> * Describe the structure you observe. Can you identify nodes with a privileged position in the network? Do you observe chains of connected nodes? Do you see any interesting group of nodes (can you guess who's involved)?

#### Create a nice visualization of the total (undirected) network:
>   * Color nodes according to the role;
>   * Scale node-size according to degree;
>   * Get node positions based on the Force Atlas 2 algorithm;
>   * Whatever else you feel like that would make the visualization nicer.

In [ ]:
#function which maps role to color
def role_to_color(role):
    if role =='Ally':
        return "green"
    elif role =='Enemy':
        return "blue"
    elif role =='Boss':
        return "red"
   

In [ ]:
#function which scales the nodes according to degree 
def node_degree_to_size(degree):
    return degree ** 1.5

In [ ]:
#function which assigns a label to nodes with a specific degree
def node_degree_to_label(degree):
    return degree > 17

In [ ]:
#list of roles
role = nx.get_node_attributes(G_undirected, 'Role')
#list of colors
#Nodes are colored according to the role
role_color = [role_to_color(r) for r in role.values()]
#list of node sizes
node_size = [node_degree_to_size(d) for k, d in dict(G_undirected.degree).items()]
#list of node labels
node_label = {k:k for k, d in dict(G_undirected.degree).items() if node_degree_to_label(d)}

In [ ]:
#function which plots the visualization of the total (undirected) network using the Force Atlas 2 algorithm
def plot_with_fa(graph, _node_colors, _node_size, _node_label):
    plt.rcParams['figure.figsize'] = [20, 15]
    forceatlas2 = ForceAtlas2(
                            # Behavior alternatives
                            outboundAttractionDistribution=True,  # Dissuade hubs
                            edgeWeightInfluence=1.0,

                            # Performance
                            jitterTolerance=7,  # Tolerance
                            barnesHutOptimize=True,
                            barnesHutTheta=1.6,

                            # Tuning
                            scalingRatio=2.0,
                            strongGravityMode=False,
                            gravity=15.0,

                            # Log
                            verbose=True)

    #node positions based on the Force Atlas 2 algorithm 
    positions = forceatlas2.forceatlas2_networkx_layout(graph, pos=None, iterations=4000)
    #draw nodes
    nx.draw_networkx_nodes(graph, positions, alpha=1, node_color=_node_colors, node_size=_node_size)
    #draw edges
    nx.draw_networkx_edges(graph, positions, edge_color=_node_colors, alpha=0.15)
    #draw labels
    nx.draw_networkx_labels(graph,positions, _node_label, font_size=10, font_color='black')
    #hide axis
    plt.title("Fig. 7. Visualization of the Total Undirected Network")
    plt.axis('off')
    plt.show()


In [ ]:
plot_with_fa(G_undirected, role_color, node_size, node_label)

Figure 7 above provides the visualization of the whole undirected graph G with 540 nodes and 1633 links. The nodes and the edges colors were colored based on the role attribute (green for allies, blue for ennemies, red for bosses), and the node size was scaled based on the node degree. 

> **Describe the structure you observe. Can you identify nodes with a privileged position in the network? Do you observe chains of connected nodes? Do you see any interesting group of nodes (can you guess who's involved)?|**

We can notice from Figure 7 that there are 3 central nodes with a privileged position in the network. These are 
Link, which was the top connected node (Ally, 396 links), Calamity Ganon (Boss, 97 links), and Hylia (Ally, 67 links). These three nodes have the highest degree centrality which scales us proportionally with their degree. 

In [ ]:
print(f'Top 3 nodes with the highest degree centrality:\n {sorted(nx.degree_centrality(G).items(), key=lambda t: t[1], reverse=True)[:3]}')

In [ ]:
print(f'Degree centrality Link: {nx.degree_centrality(G)["Link"]}')
print(f'Degree centrality Hylia: {nx.degree_centrality(G)["Hylia"]}')
print(f'Degree centrality Calamity Ganon: {nx.degree_centrality(G)["Calamity Ganon"]}')

Also, from the figure above, we can notice that there are a few chains of connected nodes, especially chains of allies and of ennemies. If we follow the blue and green nodes, we notice that most of the these chains of nodes have edges of the same color, indicating that characters have more links with other characters sharing the same role.

In terms of node size, we can notice that the smaller nodes tend to cluster around the bigger nodes. Some of the most interesting clusters are formed around Link, Calamity Ganon, Hylia, Princess Zelda, Riju, Sidon, Mipha and Bokoblin. In the Zelda universe, these are characters playing a central role in the main plot, being either friends of the main character (Sidon) or chiefs (Riju) or very common ennemies (Bokoblin) or major antagonists (Calamity Ganon), which explains why they are interconnected with the other characters. 

<!-- Node centrality 
1) degree centrality - perfectly correlated with the degree 
2) closeness centrality  = mean distance between a vertex and all the other vertices reachable from it (shortest path) - not perfectly correlated with a degree
3) betweenness - vertices that occur on many shortest paths between other vertices have higher betweenness than those that don't 
4) eigenvector centrality - measure of the importance of a node in a network; it assigns relative scores to all nodes in a network based on the principle that connections to high-scoring nodes contribute more to the score of the node in question than equal connections to high-schoring nodes -->

# Part 2: Word-clouds

Create your own version of the word-clouds (from lecture 7). For this exercise we assume you know how to download and clean text from the ZeldaWiki pages.

Here's what you need to do:
> * Create a word-cloud for each race of the [five champions of Hyrule](https://zelda.fandom.com/wiki/Champions) (i.e. Hylian, Zora, Goron, Gerudo, and Rito) according to either TC-IDF. Feel free to make it as fancy as you like. Explain your process and comment on your results.

The process decribed below indicates how the Wiki pages have been preprocessed for each character.
<ol>
<li>The extracted wiki page is read.
<li>The headers are removed using re.sub() and and the whole text is converted to lowercase.
<li>new lines ('\n') are removed using re.sub() and replaced with a space to avoid words getting pushed together later on when removing punctuation.
<li>Names consisting of multiple names (e.g. Princess Zelda) are removed using re.sub().
<li>The text is tokenized with nltk.WordPunctTokenizer().tokenize(), making each word into a list item
<li>All remaining character names are removed using if statements in a list comprehension.
<li>Punctuation is removed by only keeping tokens consisting of letters. Stopwords are also removed. All is done with if statements in a list comprehension
<li>The cleaned tokens are lemmatized with nltk.WordNetLemmatizer().lemmatize().
<li>The resulting clean text is dumped to a json file in a folder named 'preprocessed_texts'.
</ol>

In [ ]:
with ZipFile("preprocessed_texts.zip", 'r') as zipFile:
    zipFile.extractall()
    
#Please explain how we got this folders
path_ftxts = "preprocessed_texts/"

# save list of characters
characters = list(df['Name'])
unique_races = df.Race.unique()

preprocessed_tokens = []
# Fill dict of communities with a long list of all words for each community, taken from the words in their characters wikis
for char in characters:
    with open(path_ftxts+char.replace(" ","_")+".txt") as x:
        temp = json.load(x)
        preprocessed_tokens += temp
text_all = nltk.Text(preprocessed_tokens)

The process of calculating TC-IDF and plotting the word clouds for each race is described below:
<ol>
<li> Combine all words together in one long string (preprocessed_tokens)
<li> Find Term Frequency (TF) for every word within each race 
<li> Find Inverse Document Frequency (IDF) 
<li> Calculate TFIDF for all races using the TF and IDF
<li> Get TCIDF from TFIDF by un-normalizing the TF
<li> Make a list of words for each race that is replicated according to its TCIDF score
<li> Plot the wordclouds
</ol>

In [ ]:
#Calculate TFIDF for all races
#Make a dict of races
text_races = {race:[] for race in unique_races}
# Fill dict of races with a long list of all words in each wiki for every character in a race
for char in characters:
    with open(path_ftxts+char.replace(" ","_")+".txt") as x:
        text_races[df[df.Name == char]["Race"].item()] += json.load(x)
        
# Find Term Frequency (TF) for every word within each race (local search) #
TF_races = {race:[] for race in unique_races}
for race in unique_races:
    # normalize output from FreqDist
    race_to_TF = {node:count/len(text_races[race]) for node,count in FreqDist(text_races[race]).items()}
    # sort by descending values
    TF_races[race] += sorted(race_to_TF.items(), key=lambda x: x[1], reverse=True)
    TF_races[race] = dict(TF_races[race])

# Find Inverse Document Frequency (IDF) #
# number of documents N=|D|. D is the races!
numdocs = len(unique_races)
# number of documents where term t appears |{d \in D: t \in d}|
numdocs_tinD = {word:len([word for race in unique_races if word in text_races[race]]) for word in tqdm(set(text_all))}
IDF_races = {word:math.log(numdocs/numdocs_tinD[word]) for word in set(text_all)}

# calculate TFIDF for each word in each race. So a dict of races, where each element is a dict of words and their TFIDF values
TFIDF_races = {race:{word:TF_races[race][word]*IDF_races[word] for word in set(text_races[race])} for race in unique_races}

# Get TCIDF from TFIDF by un-normalizing the TF
TCIDF_races = {race:{word:tf_idf*len(text_races[race]) for word,tf_idf in race_dict.items()} for race,race_dict in TFIDF_races.items()}

In [ ]:
## Function to plot the wordclouds for the inputs in a given category. Also preps data for the WordCloud() function ##
def plot_wordclouds(Category,TCIDF,unique_races,mask=None):
    # prepare text by category for the wordcloud function. Add each word a number of times according to their TC-IDF scores
    onestring_races = {race:"" for race in unique_races}
    for race in unique_races:
        for word in TCIDF[race]:
            for _ in range(round(TCIDF[race][word])):
                onestring_races[race] += word+" "
        # Generate wordcloud object to be plotted
        wc = WordCloud(width=1500,height=750,collocations=False,background_color='white',mask=mask).generate(onestring_races[race])
        # plot each wordcloud
        plt.figure(figsize=(10,8))
        plt.imshow(wc,interpolation="bilinear")
        plt.axis('off')
        plt.title(f'{Category}: {race}', fontdict = {'fontsize' : 20})
        plt.show()

In [ ]:
# plot the wordclouds for the races of the 5 champions
unique_races_champions = ['Hylian', 'Zora', 'Goron', 'Gerudo', 'Rito']
plot_wordclouds("Race",TCIDF_races,unique_races_champions)

# Part 3: Communities and TF-IDF

Find communities and compute their associated TF-IDF (from lecture 7 and 8).

Here's what you need to do:
> * Explain the Louvain algorithm and how it finds communities in a newtork.
> * Explain how you chose to identify the communities: Which algorithm did you use? (if you did not use the Louvain method, explain how the method you have used works) 
> * Comment your results:
>   * How many communities did you find in total?
>   * Compute the value of modularity with the partition created by the algorithm.
>   * Plot the distribution of community sizes.
> * For the 5 largest communities, create TF-IDF based rankings of words in each community. 
>   * There are many ways to calculate TF-IDF, explain how you've done it and motivate your choices.
>   * List the 5 top words for each community according to TF.
>   * List the 5 top words for each community accourding to TF-IDF. Are these words more descriptive of the community than just the TF? Justify your answer.

> * **Explain the Louvain algorithm and how it finds communities in a newtork.**

The Louvain algorithm is unsupervised, meaning that it doesn't require you to define the size or number of communities beforehand. It is split up into 2 parts: Modularity optimization and community aggregation [2], which are repeated iteratively until no further improvement in modularity can be achieved.

Phase 1:

Starting with modularity optimization, the algorithm starts with a weighted network of $N$ nodes, and assigns a different community to every single node, meaning there will be as many communities as nodes.

For every node $i$, we consider the neighbouring nodes $j$ of $i$, and calculate the gain in modularity from removing $i$ from its current community and placing it into the community of $j$. This is done via the following formula.


$$\Delta Q = \left[\frac{\sum_{in}+2k_{i, in}}{2m}-\left(\frac{\sum_{tot}+k_i}{2m}\right)^2 \right] \left[\frac{\sum_{in}}{2m}-\left(\frac{\sum_{tot}}{2m}\right)^2-\left(\frac{k_i}{2m}\right)^2\right]$$
    

Here $\Delta Q$ is the gain in modularity obtained by moving node $i$ into community $C$, $\sum_{in}$ is the sum of the weights of the links inside $C$, $\sum_{tot}$ is the sum of the weights of the links incident to nodes in $C$, $k_{i}$ is the sum of the weights of the links incident to $i$, $k_{i, in}$ is the sum of the weights of the links from $i$ to nodes in $C$ and $m$ is the sum of the weights of all the links in the network. The node $i$ is then moved into the neighbouring community which gives the maximum $\Delta Q$, if a positive one exists. Else it remains in it's current community. This process is continued until no further improvements in modularity are possible. This phase only allows for local changes in communities.
    
Phase 2:

In the next phase a new network of nodes is built using the communities found in Phase 1.

These two phases are repreated iteratively until no further improvement in modularity can be achieved, whereafter the algorithm stops.
  

  > * **Explain how you chose to identify the communities: Which algorithm did you use? (if you did not use the Louvain method, explain how the method you have used works)**
  
We chose to use the Louvain algorithm for community detection. See above for a detailed description of the algorithm.

> * **Comment your results:**
>   * **How many communities did you find in total?**
>   * **Compute the value of modularity with the partition created by the algorithm.**

In [ ]:
# use read saved DiGraph G from Part 0
partition = community_louvain.best_partition(G_undirected)
print(f'Number of communities: {len(set(partition.values()))}')
communities_louvain = [[node for node in partition if partition[node] == cat] for cat in set(partition.values())]


In [ ]:
modularity_louvain = nx.algorithms.community.modularity(G_undirected,communities_louvain)
print(f'Modularity for communities by louvain-algorithm: {modularity_louvain}')

>   * **Plot the distribution of community sizes.**

In [ ]:
# Plot the distribution of community sizes
com_sizes = FreqDist(partition.values())
data = sorted(com_sizes.items(), key=lambda x:x[1])
fig,ax = plt.subplots(figsize=(8,6))

sns.barplot(x=[x for x, y in data],y=[y for x, y in data], order=[x for x, y in data])
plt.rcParams["figure.figsize"] = (10,8)
plt.xlabel('Community')
plt.ylabel('Count')
plt.title('Fig. 8. Distribution of Community Sizes')
plt.show()

Figure 8 displays the distribution of the community sizes found by the Louvain algorithm. The $x$-axis gives the different communities and the $y$-axis gives the number of nodes in a community. We can see that the majority of characters have been placed in community 2. There is also a large amount in communities 3, 4 and 11. As the Louvain algorithm has stochastic elements, the communities change everytime we calculate them.

> * **For the 5 largest communities, create TF-IDF based rankings of words in each community.**
>   * **There are many ways to calculate TF-IDF, explain how you've done it and motivate your choices.**

We calculate TF-IDF via the following. First we find the term frequency of each term $t$ in each community $d$ with
>
> $$\text{tf}(t,d) = \frac{f_{t,d}}{\sum_{t'\in d} f_{t',d}},$$

where $f_{t,d}$ is the number of times that term $t$ appears in community $d$ and $\sum_{t'\in d} f_{t',d}$ is the number of unique terms in a community. Next we calculate the inverse document frequency of term $t$ in the set of communities $D$ with
>
> $$\text{idf}(t,D) = \log\left(\frac{N}{|\{d\in D: t\in D\}|}\right),$$

where $N$ is the total number of communities and $|\{d\in D: t\in D\}|$ is the number of communities where the term $t$ appears. Finally we calculate the TF-IDF values of every term $t$ in every community $d$ given a set of communities $D$ with
>
> $$\text{tfidf}(t,d,D) = \text{tf}(t,d)\cdot\text{idf}(t,D).$$

In [ ]:
unique_coms = [com for com,_ in sorted(com_sizes.items(), key=lambda x: x[1], reverse=True)[:5]]
print(f'5 largest communities: {unique_coms}')

In [ ]:
### Calculate TFIDF for each community ###

# save list of characters according to graph and top 5 communities
characters = [x for x in partition.keys() if partition[x] in unique_coms]
# make new list of preprocessed_tokens from nodes (characters) in partition
preprocessed_tokens = []
# Make a dict of communities
text_coms = {com:[] for com in unique_coms}
# Fill dict of communities with a long list of all words for each community, taken from the words in their characters wikis
for char in characters:
    with open(path_ftxts+char.replace(" ","_")+".txt") as x:
        temp = json.load(x)
        text_coms[partition[char]] += temp
        preprocessed_tokens += temp
text_all_coms = nltk.Text(preprocessed_tokens)
# Find term frequency (TF) for each word within each community (local search)
TF_coms = {com:[] for com in unique_coms}
for com in unique_coms:
    # normalize
    com_to_tf = {node: count /len(text_coms[com]) for node, count in FreqDist(text_coms[com]).items()}
    # sort the result
    TF_coms[com] += sorted(com_to_tf.items(), key=lambda x: x[1], reverse=True)
    TF_coms[com] = dict(TF_coms[com])

# Find Inverse Document Frequency (IDF) #
# number of documents N=|D|. D is the communities!
numdocs = len(unique_coms)
# number of documents where term t appears (adjusted with +1 t avoid division by zero) |{d \in D: t \in d}|
numdocs_tinD = {word:len([word for com in unique_coms if word in text_coms[com]]) for word in tqdm(set(text_all_coms))}
# IDF = log(N/|{d \in D: t \in D}|)
IDF_coms = {word:math.log(numdocs/numdocs_tinD[word]) for word in set(text_all_coms)}

# calculate TFIDF for each community and each word in each community
TFIDF_coms = {com:{word:TF_coms[com][word]*IDF_coms[word] for word in set(text_coms[com])} for com in unique_coms}

>   * **List the 5 top words for each community according to TF.**

In [ ]:
# Find top 5 term for each of the top 5 communities
top5terms = {com:sorted(TF_coms[com].items(),key=lambda x: x[1],reverse=True)[:5] for com in unique_coms}
print(f'Top 5 terms for all commmunities by Term Frequency (word, TF):\n')
for com in unique_coms:
    print(f'Community {com}: {[(word,round(val,4)) for word,val in top5terms[com]]}')

>   * **List the 5 top words for each community accourding to TF-IDF. Are these words more descriptive of the community than just the TF? Justify your answer.**

In [ ]:
# Find top 5 TFIDF value words for each of the top 5 communities
top5_tfidf_coms = {com:sorted(TFIDF_coms[com].items(), key=lambda x: x[1],reverse=True)[:5] for com in unique_coms}
print(f'Top 5 TF-IDF value words for each community (word, TF-IDF score):\n')
for com in unique_coms:
    print(f'Community {com}: {[(word,round(tf_idf,4)) for word,tf_idf in top5_tfidf_coms[com]]}')

It can be seen when comparing the top words found with TF-IDF to the top words found with TF alone, that the words found with TF include quite a few non-descriptive words, such as "tell" and "also". These words do not represent the characters in the communities, as they are just filler words, not differentiating the characters in one community from another.

We can also see that the TF-IDF score is more descriptive than the TF alone by looking at the definition of both (see definition above). While the TF only looks at the number of times a term appears in a community,  the TF-IDF also weights this by the IDF, which is a measure of how much information the term provides to the community. So the IDF will be higher if the the term is rare across the different communities. This means the TF-IDF will be highest when a term is very frequent in a community and shows up in that community only. This further highlights why the TF-IDF score gives words that are more descriptive than when only using the TF.

# Part 4: Sentiment of communities

Analyze the sentiment of the communities (lecture 8). Here, we assume that you've successfully identified communities.  More tips & tricks can be found, if you take a look at Lecture 8's exercises.

A couple of additional instructions you will need below:
* We name each community by its three most connected characters.
* Average the average sentiment of the nodes in each community to find a community level sentiment.

Here's what you need to do (repeat these steps and report your results for **both LabMT and VADER**):
> * Calculate and store sentiment for every character
> * Create a histogram of all character's associated sentiments.
> * What are the 10 characters with happiest and saddest pages?

Now, compute the sentiment of communities: 
> * What are the three happiest communities according to the LabMT wordlist approach? What about VADER?
> * What are the three saddest communities according to the LabMT wordlist approach? What about VADER?
> * Create a bar plot showing the average sentiment of each community and add error-bars using the standard deviation for both methods. 
> * Explain the difference between the two methods and compare the results you have obtained above.
> * What is the advantage of using a rule-based method over the dictionary-based approach? 

Before we can continue with the exercise, we need to prepare the word sentiment dataframe and prepare the dialogues.

In [ ]:
# Load word sentiment dataset from [2] (Reference at the bottom)
df_sentiment = pd.read_csv(Path("Data_Set_S1.txt"), sep="	")
df_sentiment.head()[["word", "happiness_average"]]

**A couple of additional instructions you will need below:**
* **We name each community by its three most connected characters.**

In [ ]:
# Load dialogues data from web, then limit the dataframe to the characters included there.
dialogues_url = "https://raw.githubusercontent.com/SocialComplexityLab/socialgraphs2021/main/files/CharactersDialogue.json"
r = requests.get(dialogues_url)
character_to_dialogues = r.json()

df_gcc = df[df["Name"].apply(lambda name: name in G.nodes)].reset_index(drop=True)
#Create datafame which indexes the dialogues to their corresponding character
df_gcc_dialogues = df_gcc[df_gcc["Name"].apply(lambda name: name in character_to_dialogues)].reset_index(drop=True)
df_gcc_dialogues["dialogues"] = df_gcc_dialogues["Name"].apply(lambda name: character_to_dialogues[name])

f"Out of {len(df_gcc)} characters in the greatest component subgraph, we only analyze {len(df_gcc_dialogues)} found in dialogues."

In [ ]:
#Undirected graph
G_undirected = G.to_undirected()
#Identify partitions using the Louvain algorithm
node_to_partition_id = community_louvain.best_partition(G_undirected)

In [ ]:
#Function to find 3 most connected nodes within a partition and concatenating them to obtain a partition name
def partition_to_top3_names(G, node_to_partition_id, partition_id):
    
    nodes_in_partition = [node for node in G.nodes() 
                          if node_to_partition_id[node] == partition_id]
    sub_G = G.subgraph(nodes_in_partition)
    node_to_connectivity = [(node, sub_G.degree(node)) for node in sub_G.nodes()]
    top3_nodes = sorted(node_to_connectivity, key=lambda t: t[1], reverse=True)[:3]
    return ", ".join([n for n, _ in top3_nodes])
    
# Assign partition name to each partition_id
partition_id_to_name = {partition_id: partition_to_top3_names(G_undirected, node_to_partition_id, partition_id) 
                        for partition_id in set(node_to_partition_id.values())}


In [ ]:
# Name community by its three most connected characters in df_gcc
# Dataframe which stores sentiment for every character
df_gcc_dialogues["community"] = df_gcc["Name"].apply(lambda name: partition_id_to_name[node_to_partition_id[name]])
df_gcc_dialogues.head()

* **Average the average sentiment of the nodes in each community to find a community level sentiment.**

In [ ]:
# Function which preprocesses raw dialogues before analysis with LabMT sentiment analysis method
wordnet_lemmatizer = WordNetLemmatizer()
punctuations = "?:!.,;"

def preprocess(sentences: List[str]) -> List[str]: # List of sentences to lists of processed words
    sentences = " ".join(sentences)
    sentence_words = nltk.word_tokenize(sentences)
    words_lemmatized = [wordnet_lemmatizer.lemmatize(w, pos='v').lower() 
                        for w in sentence_words 
                        if w not in punctuations]
    return words_lemmatized

#Example
print(f"Example: {preprocess(['He is running very fast!', 'Thats impressive'])}")

In [ ]:
#Function which uses inner join between words from the text we analyze and the words in the word sentiment dataframe
#by matching each word to its sentiment score.
#Words with no assigned sentiment value are not to be taken into account
def compute_sentiment_LabMT(words):
    words_series = pd.Series(words).rename('word')
    sentiments = df_sentiment.merge(words_series, how='inner', on="word")
    return sentiments["happiness_average"].mean()

#Example
example1 = ["I am incredibly happy to be here, my dear"]
example2 = ["You are the worst"]
print(example1, compute_sentiment_LabMT(preprocess(example1)))
print(example2, compute_sentiment_LabMT(preprocess(example2)))

In [ ]:
#Preprocessing of the raw dialogues in LabMT
df_gcc_dialogues["words"] = df_gcc_dialogues["dialogues"].apply(preprocess)
#Compute the average sentiment for each dialogue in the LabMT dataset
df_gcc_dialogues["sentiment_LabMT"] = df_gcc_dialogues["words"].apply(compute_sentiment_LabMT)

In [ ]:
analyzer = SentimentIntensityAnalyzer()
#Function which computes the average sentiment in dialogue sentences
def compute_sentiment_VADER(dialogues):
    return np.mean([analyzer.polarity_scores(sentence)['compound'] for sentence in dialogues])

#Compute the average sentiment for each dialogue in the VADER dataset
df_gcc_dialogues["sentiment_VADER"] = df_gcc_dialogues["dialogues"].apply(compute_sentiment_VADER)

In [ ]:
#Dataframe for the average sentiment in Vader and LabMT for each community
community_to_avg_sentiments = [{"community": community,
                                "avg VADER sentiment": round(df_comm["sentiment_VADER"].mean(), 3),
                                "avg LabMT sentiment": round(df_comm["sentiment_LabMT"].mean(), 3) }   
                                for community, df_comm 
                                in df_gcc_dialogues.groupby("community")]

community_to_avg_sentiments = pd.DataFrame(community_to_avg_sentiments)
community_to_avg_sentiments

**Here's what you need to do (repeat these steps and report your results for both LabMT and VADER):**
> * **Calculate and store sentiment for every character**

In [ ]:
#Dataframe which stores sentiment for every character (calculated above)
df_gcc_dialogues.head()

> * **Create a histogram of all character's associated sentiments.**

In [ ]:
#Histogram of All Character's Associated Sentiments (VADER)
df_gcc_dialogues["sentiment_VADER"].plot.hist()
plt.rcParams["figure.figsize"] = (10,8)
plt.title("Fig. 9. Histogram of All Character's Associated Sentiments (VADER)")
plt.ylabel("Count")
plt.xlabel("Sentiment score")

We notice that in Fig. 9, the values of the sentiment score lies between -1 and 1 and they follow a Gaussian distribution around 0.21. We can assume that according to the VADER sentiment score, the dialogues are expected to be neutral.

In [ ]:
#Histogram of All Character's Associated Sentiments (LabMT)
plt.rcParams["figure.figsize"] = (10,8)
plt.title("Fig. 10. Histogram of All Character's Associated Sentiments (LabMT)")
df_gcc_dialogues["sentiment_LabMT"].plot.hist()
plt.ylabel("Count")
plt.xlabel("Sentiment score")

In Fig. 10, we notice that the values of sentiment score for LabMT method lie between 5.18 and 5.9. The histogram follows a Gaussian distribution, with most values centered around 5.5 (neutral). 

> * **What are the 10 characters with happiest and saddest pages?**

In [ ]:
pd.options.display.max_colwidth = 150
#top 10 characters with happiest and saddest pages in Vader
vader_happy= df_gcc_dialogues.nlargest(10, "sentiment_VADER")[["Name",  "sentiment_VADER"]]
vader_sad = df_gcc_dialogues.nsmallest(10, "sentiment_VADER")[["Name","sentiment_VADER"]]
#top 10 characters with happiest and saddest pages in LabMT
labmt_happy = df_gcc_dialogues.nlargest(10, "sentiment_LabMT")[["Name", "sentiment_LabMT"]]
labmt_sad = df_gcc_dialogues.nsmallest(10, "sentiment_LabMT")[["Name","sentiment_LabMT"]]

print(f'Top 10 characters with happiest pages in VADER:\n {list(vader_happy.Name)}')
print(f'Top 10 characters with saddest pages in VADER:\n {list(vader_sad.Name)}')
print(f'Top 10 characters with happiest pages in LabMT:\n {list(labmt_happy.Name)}')
print(f'Top 10 characters with saddest pages in LabMT:\n {list(labmt_sad.Name)}')

**Now, compute the sentiment of communities:**
> * **What are the three happiest communities according to the LabMT wordlist approach? What about VADER?**

In [ ]:
#Function which returns the three happiest or saddest communities by selected method
def top_3_communities_by_sentiment(df, happiest: bool, sentiment_source: str):
    community_sentiments = [(community, round(df_comm[sentiment_source].mean(), 3))  
                            for community, df_comm in df.groupby("community")]
    return sorted(community_sentiments, key=lambda t: t[1], reverse=happiest)[:3]

In [ ]:
top_labmt =top_3_communities_by_sentiment(df_gcc_dialogues, happiest=True, sentiment_source="sentiment_LabMT")
top_vader = top_3_communities_by_sentiment(df_gcc_dialogues, happiest=True, sentiment_source="sentiment_VADER")
print('The top 3 happiest communities in LabMT:\n', [k[0] for k in top_labmt])
print('The top 3 happiest communities in Vader:\n', [k[0] for k in top_vader])

> * **What are the three saddest communities according to the LabMT wordlist approach? What about VADER?**

In [ ]:
top_labmt =top_3_communities_by_sentiment(df_gcc_dialogues, happiest=False, sentiment_source="sentiment_LabMT")
top_vader = top_3_communities_by_sentiment(df_gcc_dialogues, happiest=False, sentiment_source="sentiment_VADER")
print('The top 3 saddest communities in LabMT:\n', [k[0] for k in top_labmt])
print('The top 3 saddest communities in Vader:\n', [k[0] for k in top_vader])

> * **Create a bar plot showing the average sentiment of each community and add error-bars using the standard deviation for both methods.**

In [ ]:
#Function to plot the average sentiment of each community
def show_bar_plot(df, sentiment_source, title, ylim=None):
    df.groupby("community")\
            .agg(sentiment=(sentiment_source, np.mean), std=(sentiment_source, np.std))\
            .sort_values("sentiment")\
            .plot.bar(y="sentiment", ylim=ylim,yerr="std")
    plt.rcParams['figure.figsize'] = [10, 8]
    plt.legend(loc='upper left', title=sentiment_source)
    plt.title(title)
    plt.show()

In [ ]:
# For readability, the bar plot will start at 5
show_bar_plot(df_gcc_dialogues, sentiment_source="sentiment_LabMT", ylim=(5.2, 5.9), title="Fig. 11. Bar plot of the average sentiment for LabMT")

Figure 11 displays the average sentiment in each community based on the LabMT dataset. We notice that the most negative community is represented by three enemies (Ashai, Pearle, Dalia), which indicates that their conversations are representative of their roles as villains. Also, we can notice that the most positive community is represented by characters who Link's helpers (Spinch (Horse) and Spinch (Hylian). 

Overall, we can see that deviation for this method is quite high when comparing the leftmost and rightmost community.

In [ ]:
show_bar_plot(df_gcc_dialogues, sentiment_source="sentiment_VADER", title="Fig. 13. Bar plot of the average sentiment for VADER")

Figure 12 displays the average sentiment in each community based on the VADER dataset. We notice that for all the communities the average sentiment is above 0, meaning that it is in the neutral to positive spectrum. 

> * **Explain the difference between the two methods and compare the results you have obtained above.**

Word-level sentiment is a score describing the emotional connotations of it. Dodds et. al [3] describes how these scores are computed. In short, authors have extracted the 50k most common words in tweets, divided them into chunks of 10 and paid people at Amazon's Mechanical Turk to rank/sort the words based on their view of how positive these words are. The resulting indexes of these words are later used as a score of the positivity.

In this exercise there are two methods implemented, which are refered to as LabMT and VADER. The LabMT method is implemented here only based on the sentiment score of each of the words in the text, as an arithmetic average of each word's score. 

On the other hand, VADER (Valence Aware Dictionary and Sentiment Reasoner) is a rule-based method, but it also leverages on the previously mentioned word-level sentiment scores. However, it uses concepts such as boosters and negators to scale and flip the sentiments of the words accordingly. 

Summing up, the difference between these methods is that LabMT only takes into consideration the sentiment of each word, while VADER attempts to interpret the sentence as a whole, showcasing how words and punctuation interact with each other, 

In order to do a more quantitative analysis, first, let us look how correlated the scores of both the methods are:

In [ ]:
round(df_gcc_dialogues["sentiment_VADER"].corr(df_gcc_dialogues["sentiment_LabMT"]), 3)

We notice that there is a positive, however not very significant correlation between these scores.

Another way of comparing these two sentiment-mining methods is by treating their results as a ranking. As they both describe the same dialogues, we can use a method called Rank-biased Overlap (RBO) described by Webber et al [4].

In [ ]:
vader_rank = list(df_gcc_dialogues.sort_values("sentiment_VADER")["Name"])
labmt_rank = list(df_gcc_dialogues.sort_values("sentiment_LabMT")["Name"])

round(rbo.RankingSimilarity(vader_rank, labmt_rank).rbo(), 3)

The results from Rank-biased Overlap (RBO) are somewhat similar, but not to a great degree. (Score of 0 would signify no similarity, 1 would signify these ranking are the same.)

Using the summaries established previously (top10 saddest, happiest characters) we can find out how many of the extreme scores are consistent across methods:

In [ ]:
last10_labmt = set(df_gcc_dialogues.nsmallest(10, "sentiment_LabMT")["Name"])
last10_vader = set(df_gcc_dialogues.nsmallest(10, "sentiment_VADER")["Name"])
intersection = last10_labmt.intersection(last10_vader)
len(intersection), intersection

In [ ]:
top10_labmt = set(df_gcc_dialogues.nlargest(10, "sentiment_LabMT")["Name"])
top10_vader = set(df_gcc_dialogues.nlargest(10, "sentiment_VADER")["Name"])
intersection = top10_labmt.intersection(top10_vader)
len(intersection), intersection

Again, we find that both the methods are moderately similar. The same findings we got from the top happiest and saddest communities comparison: in the 3 saddest communities from each method ('Aya', 'Kenyo', 'Kotta', 'Palme', 'Kotta', 'Princess Zelda') only two were the same: 'Kotta' and 'Palme'. For the 3 happiest from each method ('Kinov', 'Narah', 'Kula', 'Canni', 'Breen', 'Toren') the similarity is greater, as both 'Kula', 'Toren' were present.

> * **What is the advantage of using a rule-based method over the dictionary-based approach?**

The advantage of using the rule-based method over the dictionary-based approach is the quality of the resulting scores for whole texts. We see that VADER does a much better job in cases that require a more "holistic" approach.

Let us consider the sentence **"I do not like what I see!"**.

In [ ]:
print(f'Sentiment score with VADER:  {compute_sentiment_VADER(["I do not like what I see!"])}')

In [ ]:
print(f'Sentiment score with LabMT:  {compute_sentiment_LabMT(preprocess(["I do not like what I see!"]))}')

We find that while VADER manages to notice that the words "not" and "like" interact in a way which makes the whole sentence negative, LabMT failes at it, giving it a score of 5.65, which is quite high compared to other LabMT scores.


# References

[1] Barabasi A. "Network Science". http://networksciencebook.com/ 

[2] V. D. Blondel, J.-L. Guillaume, R. Lambiotte and E. Lefebvre, "Fast unfolding of communities in large networks," J. Stat. Mech. (2008) P10008, p. 12, 2008.

[3] Dodds, Peter Sheridan, et al. "Temporal patterns of happiness and information in a global social network: Hedonometrics and Twitter." PloS one 6.12 (2011): e26752.

[4] William Webber, Alistair Moffat, and Justin Zobel. 2010. A similarity measure for indefinite rankings. ACM Trans. Inf. Syst. 28, 4, Article 20 (November 2010), 38 pages. DOI:https://doi.org/10.1145/1852102.1852106
